In [ ]:
#Installing necessary dependences:
#!pip install -q keras-nlp


In [ ]:
#!pip list

In [ ]:
#imports:
import tensorflow as tf
import tensorflow_text as tf_text
import keras
import pandas as pd
import numpy as np
import os
import re
import string
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, MultiHeadAttention, Softmax, TextVectorization, LayerNormalization
from keras import Model, Input
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [ ]:
print(tf.__version__)

In [ ]:
print(tf.test.is_gpu_available()) 
print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import keras_nlp

In [ ]:
#tf.compat.v1.disable_eager_execution()

In [ ]:
#Preprocessing Data
filepath = "./Datasets"
filepath = re.sub(r'\\', '/', filepath)
print(filepath)
X1 = pd.read_csv(filepath + "/Tatobea Project" + '/'+ "spa" +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
X1 = X1[X1.notnull().all(axis = 1)]

In [ ]:
print(X1)

In [ ]:
X1['Translated'] = "<ES> " + X1['Translated']

In [ ]:
print(X1)

In [ ]:
train_x = pd.DataFrame()

In [ ]:
for d in [x[0] for x in os.walk(filepath + "/DGT-TM")]:
    print(d)
    print(d[-6:])
    #Ignores TMX files
    if d[-1] == ")" or d[-6:] == "DGT-TM":
        continue
    #This code actually works, however due to the dataset it takes 1+ hour to run.
    #Because of this, at least for now I'll be restricting it to just the EN-ES folder.
    te = True
    if d[-5:] != "EN-ES":
        te = False
        continue
    
    fp = re.sub(r'\\', '/', d)
    E = fp.split("/")[-1].split("-")[0]
    print(E)
    T = fp.split("/")[-1].split("-")[1]
    print(T)
    print(d + '/'+ E + "-" + T +'.txt')
    
    temp = pd.read_csv(d + '/'+ E + "-" + T +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
    temp = temp[temp.notnull().all(axis = 1)]
    temp['Translated'] = "<" + T + "> " + temp['Translated']
    
    print(temp.head())
    if te:
        X1 = pd.concat([X1, temp])
    else:
        train_x = pd.concat([train_x, temp])


In [ ]:
Y1 = pd.DataFrame()
Y1['Translated'] = X1.pop("Translated")
Y1['Translated'] = "[start] " + Y1['Translated'] + " [end]"

In [ ]:
print(Y1)

In [ ]:
train_x1, test_x, train_y1, test_y = train_test_split(X1, Y1, test_size=0.5, random_state=42)

In [ ]:
#val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=42)

In [ ]:
try:
    train_y = train_x.pop("Translated")
except:
    #train_x doesn't have any portions, so we'll just describe it as a normal Dataframe
    train_y = pd.DataFrame()

In [ ]:
train_x = pd.concat([train_x, train_x1])

In [ ]:
train_y = pd.concat([train_y, train_y1])

In [ ]:
del train_x1
del train_y1
del X1
del Y1
del temp

In [ ]:
print(train_x)
print(train_y)
print(test_x)
print(test_y)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
train_pairs = pd.concat([train_x['English'], train_y['Translated']], axis=1).values.tolist()
val_pairs = pd.concat([test_x['English'], test_y['Translated']], axis=1).values.tolist()
print(train_pairs[0])
print(train_pairs[1])

In [ ]:
#train_x = train_x.astype(str)
#test_x = test_x.astype(str)

In [ ]:
#X = pd.concat([train_x, test_x])
#Y = pd.concat([train_y, test_y])
#Y_SP = Y[Y['Translated'].str.contains("<ES>")]

In [ ]:
file = open('engvocab.txt', 'r', encoding="utf-8")
input_vocab = file.read().split("\n")
file.close()
input_vocab_size = len(input_vocab)

file = open('spavocab.txt', 'r', encoding="utf-8")
output_vocab = file.read().split("\n")
file.close()
output_vocab_size = len(output_vocab)

In [ ]:
del input_vocab[-1]
del output_vocab[-1]

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=input_vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    vocabulary=input_vocab,
)
spa_vectorization = TextVectorization(
    max_tokens=output_vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
    vocabulary=output_vocab,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
#eng_vectorization.adapt(train_eng_texts)
#spa_vectorization.adapt(train_spa_texts)

In [ ]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
#STOP

In [ ]:
#Actual Model:
#For now, let's start by creating an exact copy of the attention is all you need model.
#I tried for a long time to make a model using only the Keras Functional API,
#but it seems like I will need to create some custom layers in order to succeed.


In [ ]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
    ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
        super(EncoderLayer, self).__init__()


        self.embed_dim = d_model
        self.dense_dim = dff
        self.num_heads = num_attention_heads
        
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        
        self.dense_proj = point_wise_feed_forward_network(d_model, dff)

        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()


        # Dropout for the point-wise feed-forward network.
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    
        self.supports_masking = True

    def call(self, inputs, mask=None):


        attention_output = self.attention(query=inputs, value=inputs, key=inputs)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
               *,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
        super(DecoderLayer, self).__init__()

        # Masked multi-head self-attention.
        self.attention_1 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )
        # Multi-head cross-attention.
        self.attention_2 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )

        # Point-wise feed-forward network.
        self.dense_proj = point_wise_feed_forward_network(d_model, dff)

        # Layer normalization.
        self.layernorm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.add = tf.keras.layers.Add()  # instead of `+` to preserve mask

        # Dropout for the point-wise feed-forward network.
        #self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, encoder_outputs, mask=None):
        # The encoder output shape is `(batch_size, input_seq_len, d_model)`.

    
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, use_causal_mask=True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(self.add([out_2, proj_output]))


In [ ]:
#Original hyperparameters
num_layers = 4
embed_dim = 256
latent_dim = 512
num_heads = 8
dropout_rate = 0.1
epochs=50
batch_size = 128
sequence_length = 256

In [ ]:
#Dataset_train = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor)).batch(batch_size)
#Dataset_test = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor)).batch(batch_size)

In [ ]:
#tf.compat.v1.disable_eager_execution()

In [ ]:
#print(Dataset_train)
#print(Dataset_test)

In [ ]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, num_layers, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.enc_layers = [
        EncoderLayer(
          d_model=embed_dim,
          num_attention_heads=num_heads,
          dff=dense_dim,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
        
        self.supports_masking = True

    def call(self, inputs, mask=None):
        x = inputs
        for i in range(self.num_layers):
            #x = self.enc_layers[i](x, mask)
            x = self.enc_layers[i](x)
        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config






In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


In [ ]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, num_layers,  dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.dec_layers = [
        DecoderLayer(
          d_model=embed_dim,
          num_attention_heads=num_heads,
          dff=latent_dim,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
        
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        x = inputs
        for i in range(self.num_layers):
            #x = self.dec_layers[i](x, mask)
            x = self.dec_layers[i](x, encoder_outputs)
        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [ ]:
print(input_vocab_size)

In [ ]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, input_vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads, num_layers, dropout_rate)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, output_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads, num_layers, dropout_rate)(x, encoded_seq_inputs)
x = tf.keras.layers.Dropout(0.5)(x)
decoder_outputs = tf.keras.layers.Dense(output_vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
#tf.compat.v1.enable_eager_execution()

In [ ]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
#with tf.device('/GPU:0'):
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
print(tf.__version__)

In [ ]:
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

In [ ]:
#i = Input(shape=(max_length))

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#TLayer = TextVectorization(output_sequence_length = max_padding_size, vocabulary = m, output_mode = "int")
#inp = TextVectorization(output_sequence_length = max_padding_size)

In [ ]:
#inp = TLayer(i)

In [ ]:
#print(inp)

In [ ]:
#inp.adapt(train_x)

In [ ]:
#dir(inp)

In [ ]:
#print(inp.get_vocabulary())

In [ ]:
#f = open("textvectorvocap.txt", "w", encoding="utf-8")
#f.write("*".join(inp.get_vocabulary()))
#f.close()

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#print(m == inp.get_vocabulary())

In [ ]:
#print(inp.vocabulary_size())

In [ ]:
#train_x_vec = inp.predict(train_x)
#test_x_vec = inp.predict(test_x)

In [ ]:
#print(i.__call__())

In [ ]:
#em = Embedding(input_dim = max_length, output_dim = 512)(inp)

In [ ]:
#posen = keras_nlp.layers.SinePositionEncoding()(em)

In [ ]:
#mh1 = MultiHeadAttention(num_heads=3, key_dim=512, value_dim=512)

In [ ]:
#mho1 = mh1(posen, posen, posen)

In [ ]:
#no1 = LayerNormalization()
#ou1 = no1(mho1+posen)

In [ ]:
#fe1 = Dense(512)(ou1)
#ou2 = no1(ou1+fe1)

In [ ]:
#lin = Dense(tar_vocab_length)(ou2)

In [ ]:
#s = Softmax()(lin)

In [ ]:
#model = Model(inputs=i, outputs=s)
#print(model.summary())

In [ ]:
#Need to figure out metrics for language translation